현재 노트북을 제외한 모든 주피터 노트북에서 상단의 `Kernel > ShudDown Kernel`을 하신 후에 실습을 다시 시작하세요.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 경로 설정
base_model_path = "NCSOFT/Llama-VARCO-8B-Instruct"
adapter_path = "./llama-3-8b-rag-ko/checkpoint-285"
merged_model_path = "./output_dir"

# 디바이스 설정
device_arg = {"device_map": "auto"}

# 베이스 모델 로드
print(f"Loading base model from: {base_model_path}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)

# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
model = PeftModel.from_pretrained(base_model, adapter_path, **device_arg)
model = model.merge_and_unload()

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 저장
print(f"Saving merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("✅ 모델과 토크나이저 저장 완료")

Loading base model from: NCSOFT/Llama-VARCO-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading and merging PEFT from: ./llama-3-8b-rag-ko/checkpoint-285
Saving merged model to: ./output_dir
✅ 모델과 토크나이저 저장 완료


In [8]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk

test_dataset = load_from_disk('test_dataset')

In [10]:
model = 'NCSOFT/Llama-VARCO-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model)

prompt_lst = []
label_lst = []

for messages in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    input = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[0] + '<|start_header_id|>assistant<|end_header_id|>\n'
    label = text.split('<|start_header_id|>assistant<|end_header_id|>\n')[1].split('<|eot_id|>')[0]
    prompt_lst.append(input)
    label_lst.append(label)

In [14]:
eos_token = tokenizer("<|eot_id|>",add_special_tokens=False)["input_ids"][0]

def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()

In [15]:
model_id = './output_dir'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", low_cpu_mem_usage=True, torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


    response:
검색 결과에는 "방콕 셧다운"이라는 용어를 처음 누가 사용했는지에 대한 정보를 찾을 수 없습니다.
    label:

검색 결과에는 '방콕 셧다운'이라는 용어를 처음 사용한 사람이 누구인지에 대한 정보가 없습니다.
--------------------------------------------------
    response:
스뮈츠가 크리칭거 장군에게 가기 전 마지막으로 전투를 벌였던 지역은 서부 트란스발입니다. 스뮈츠는 트란스발에서 500명의 게릴라를 지휘하는 쿠스 드 라 레이 장군의 지휘를 받게 되었고, 그와 함께 여러 게릴라전에서 활약하여 영국군을 격퇴했습니다. 그러나 드 라 레이 장군이 서부 트란스발로 가면서 스뮈츠는 300명의 게릴라들을 이끌게 되었습니다. 이후 스뮈츠는 영국군의 청야전술로 인해 보어인 농장들이 파괴되고 식량 부족으로 인해 게릴라 참여를 거절해야 했지만, 결국 스파르타와 아테네 사이의 적대 관계로 인해 펠로폰네소스 전쟁으로 이어지게 되었습니다 [[ref3]].
    label:

검색 결과에는 스뮈츠가 크리칭거 장군에게 가기 전 마지막으로 전투를 벌였던 지역을 찾을 수 없습니다.
--------------------------------------------------
    response:
검색 결과에는 일연이 삼국유사를 저술하면서 어떤 자료들을 주로 참고했는지에 대한 정보를 찾을 수 없습니다.
    label:

검색 결과에 따르면, 일연이 《삼국유사》를 저술하는 데 어떤 자료들을 참고했는지에 대한 정보는 확인되지 않습니다.

=> 검색 결과에는 일연이 《삼국유사》를 저술하면서 참고한 자료들에 대한 언급이 없습니다.
--------------------------------------------------
    response:
검색 결과에는 서울 행복주택 입주자가 되기 위해 필요한 서류를 찾을 수 없습니다.
    label:

검색 결과에는 서울 행복주택 입주자가 제출해야 하는 서류에 대한 

In [16]:
from huggingface_hub import HfApi
api = HfApi()

username = "iamjoon"

In [17]:
MODEL_NAME = 'llama-3-8b-rag-ko-checkpoint-285'

In [18]:
api.create_repo(
    token="hf_여러분의 Key 값",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token="hf_여러분의 Key 값",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir",
)

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iamjoon/llama-3-8b-rag-ko-checkpoint-285/commit/98cefe39bc38e0562a292dac184da497caa88ff7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='98cefe39bc38e0562a292dac184da497caa88ff7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iamjoon/llama-3-8b-rag-ko-checkpoint-285', endpoint='https://huggingface.co', repo_type='model', repo_id='iamjoon/llama-3-8b-rag-ko-checkpoint-285'), pr_revision=None, pr_num=None)